# <b><center>Feature Engineering</center></b>

In [1]:
# # ! pip install tensorflow
# # ! pip install keras
# ! pip install -q keras
# ! pip install -q tensorflow
# ! pip install pandas

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
# Load the VGG16 model
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df__bk_rat = pd.read_csv('/Users/royzalta/Downloads/archive/Book reviews/Book reviews/BX-Book-Ratings.csv', sep=';', encoding="latin-1")
df_bk_dt = pd.read_csv('/Users/royzalta/Downloads/archive/Book reviews/Book reviews/BX_Books.csv', sep=';', encoding="latin-1")
df_usr_dt = pd.read_csv('/Users/royzalta/Downloads/archive/Book reviews/Book reviews/BX-Users.csv', sep=';', encoding="latin-1")
prepo_df = pd.read_csv('/Users/royzalta/Downloads/archive/Books Data with Category Language and Summary/Preprocessed_data.csv', sep=';', encoding="latin-1")

In [ ]:
df_bk_dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271379 non-null  object
 1   Book-Title           271379 non-null  object
 2   Book-Author          271377 non-null  object
 3   Year-Of-Publication  271379 non-null  int64 
 4   Publisher            271377 non-null  object
 5   Image-URL-S          271379 non-null  object
 6   Image-URL-M          271379 non-null  object
 7   Image-URL-L          271379 non-null  object
dtypes: int64(1), object(7)
memory usage: 16.6+ MB
